# Data Deduplication Lab - Getting Started

Welcome to the Data Deduplication Lab! This notebook will help you get started with the lab exercises.

## Learning Objectives

By the end of this lab, you will:
- Understand different deduplication strategies
- Be able to process large datasets efficiently with Spark
- Know how to use approximate methods for memory-efficient operations
- Understand file-level vs record-level deduplication

## Prerequisites

- Access to Cloudera AI Workbench
- Basic Python knowledge
- Familiarity with data processing concepts

## Setup

First, let's verify that we have access to the necessary files and Spark is available.

**Important**: Make sure you have uploaded the following files to your Cloudera AI Workbench project:
- `deduplicate_spark.py` (in the project root)
- `generate_dataset.py` (in the project root)
- `bloom_filter_hyperloglog.py` (in the project root)
- `bloom_filter_file_deduplication.py` (in the project root)

The notebook will automatically add the project root to the Python path so these modules can be imported.


In [ ]:
# Add project root to Python path so we can import modules
import sys
import os

# Find project root - try multiple methods
current_dir = os.getcwd()
project_root = None

# Method 1: If we're in notebooks/, go up one level
if 'notebooks' in current_dir:
    project_root = os.path.dirname(current_dir)
    print(f"Found project root (from notebooks/): {project_root}")
# Method 2: Check if deduplicate_spark.py is in current directory
elif os.path.exists(os.path.join(current_dir, 'deduplicate_spark.py')):
    project_root = current_dir
    print(f"Found project root (deduplicate_spark.py in current dir): {project_root}")
# Method 3: Search up directories
else:
    test_dir = current_dir
    for _ in range(5):  # Try up to 5 levels
        if os.path.exists(os.path.join(test_dir, 'deduplicate_spark.py')):
            project_root = test_dir
            print(f"Found project root (searched up): {project_root}")
            break
        parent = os.path.dirname(test_dir)
        if parent == test_dir:  # Reached filesystem root
            break
        test_dir = parent
    
    # Fallback to current directory
    if project_root is None:
        project_root = current_dir
        print(f"Using current directory as project root: {project_root}")

# Add project root to path if not already there
if project_root and project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"✓ Added to Python path: {project_root}")

# Change to project root for file operations
os.chdir(project_root)
print(f"✓ Changed working directory to: {project_root}")

# Check if Spark is available
try:
    from pyspark.sql import SparkSession
    print("✓ PySpark is available")
except ImportError:
    print("✗ PySpark not found. Please ensure Spark is installed.")

# Check if our deduplication module is available
try:
    from deduplicate_spark import create_spark_session
    print("✓ deduplicate_spark module is available")
except ImportError as e:
    print("✗ deduplicate_spark module not found.")
    print(f"  Current working directory: {os.getcwd()}")
    print(f"  Project root used: {project_root}")
    print(f"  Python path (first 3): {sys.path[:3]}")
    print(f"  Error: {e}")
    print("\n  Troubleshooting:")
    print("  1. Ensure deduplicate_spark.py is in the project root")
    print("  2. Check that the file is uploaded to Cloudera AI Workbench")
    print("  3. Verify the file is in the same directory as this notebook's parent")

# Check Python version
print(f"\n✓ Python version: {sys.version}")


## Create Spark Session

The `create_spark_session()` function automatically detects if you're running in Cloudera AI Workbench and configures Spark appropriately.


In [ ]:
from deduplicate_spark import create_spark_session

# Create Spark session (automatically configured for Cloudera)
spark = create_spark_session("DeduplicationLab")

print(f"Spark version: {spark.version}")
print(f"Spark master: {spark.sparkContext.master}")
print("✓ Spark session created successfully!")


## Generate Sample Data

Let's generate a sample dataset with duplicates to work with. We'll create a CSV file with 1000 records.


In [ ]:
import subprocess
import os

# Ensure we're in project root (should already be set from previous cell)
# Generate sample data in project root's data/ directory
data_dir = os.path.join(project_root, "data")
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"✓ Created data directory: {data_dir}")

# Generate dataset with 1000 records
output_file = os.path.join(data_dir, "redundant_data.csv")
script_path = os.path.join(project_root, "generate_dataset.py")

print(f"Running: python {script_path} 1000 {output_file}")
print(f"Working directory: {os.getcwd()}")

result = subprocess.run(
    ["python", script_path, "1000", output_file],
    cwd=project_root,  # Ensure we run from project root
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✓ Sample data generated successfully!")
    if result.stdout:
        print(result.stdout)
else:
    print("✗ Error generating data:")
    print(result.stderr)
    print(f"\nTroubleshooting:")
    print(f"  - Script path: {script_path}")
    print(f"  - Script exists: {os.path.exists(script_path)}")
    print(f"  - Current directory: {os.getcwd()}")

# Check if file was created
if os.path.exists(output_file):
    file_size = os.path.getsize(output_file)
    print(f"\n✓ File created: {output_file} ({file_size:,} bytes)")
else:
    print(f"\n✗ File not found at: {output_file}")


## Load and Inspect Data

Let's load the data into a Spark DataFrame and take a look at it.


In [ ]:
# Read the CSV file from project root's data/ directory
data_file = os.path.join(project_root, "data", "redundant_data.csv")
df = spark.read.csv(data_file, header=True, inferSchema=True)

# Show basic information
print(f"Total records: {df.count():,}")
print(f"Columns: {', '.join(df.columns)}")
print("\nFirst 10 records:")
df.show(10, truncate=False)

# Show schema
print("\nSchema:")
df.printSchema()


## Check for Duplicates

Let's see how many duplicates we have in our dataset.


In [ ]:
from pyspark.sql import functions as F

# Count total records
total_count = df.count()

# Count unique records based on name and email
unique_count = df.select("name", "email").distinct().count()

# Calculate duplicates
duplicates = total_count - unique_count
duplicate_rate = (duplicates / total_count * 100) if total_count > 0 else 0

print(f"Total records: {total_count:,}")
print(f"Unique records (by name+email): {unique_count:,}")
print(f"Duplicate records: {duplicates:,}")
print(f"Duplicate rate: {duplicate_rate:.2f}%")


## Next Steps

Now that you have your data loaded, you're ready to start the lab exercises:

1. **Exercise 1**: Basic Deduplication - `01_Basic_Deduplication.ipynb`
2. **Exercise 2**: Compare Methods - `02_Compare_Methods.ipynb`
3. **Exercise 3**: Approximate Methods - `03_Approximate_Methods.ipynb`
4. **Exercise 4**: File-Level Deduplication - `04_File_Level_Deduplication.ipynb`

## Cleanup

When you're done, remember to stop the Spark session:


In [ ]:
# Stop Spark session when done
spark.stop()
print("✓ Spark session stopped")
